# Exercise: Exploratory Data Analysis with Pandas
## Instructions
1. Data Collection
    - Search and collect data on the following: (1993 - 2023)
        - Israel interest tares
        - US interest rates
        - USD to ILS exchange rates
    - For each dataset, create a pandas DataFrame with the following columns:
        - Year
        - Month
        - Current_Rate (Interest rate or exchange rate for that month)
        - Change_From_Last_Month (Difference in Current_Rate from the previous month)
        - Optional: You may include additional columns if you find them useful (e.g., Change_Percentage, Date).



In [67]:
import pandas as pd
import numpy as np

In [68]:
# your code here
def process_df(file_path, rate_column):
    df = pd.read_csv(file_path)
    df['DATE'] = pd.to_datetime(df['DATE'])
    df['Year'] = df['DATE'].dt.year
    df['Month'] = df['DATE'].dt.month
    df['Change_From_Last_Month'] = df[rate_column].diff()
    return df[['Year', 'Month', rate_column, 'Change_From_Last_Month']].rename(columns={rate_column: 'Current_Rate'})

fed_funds_df = process_df('./FEDFUNDS.csv', 'FEDFUNDS')

conversion_ratio_df = process_df('./conversion_ratio.csv', 'CCUSSP02ILM650N')

boi_interest_rate_df = process_df('./bank_of_Israel_interest_rate.csv', 'IR3TIB01ILM156N')


## Part A: Single DataFrame Analysis 
1. Display the first 5 rows of the US interest rates DataFrame.

2. Calculate the average Israeli interest rate over the period covered by your data.

3. Find the month and year when the USD to ILS exchange rate had the highest increase from the previous month.

4. Determine the total number of months where the US interest rate decreased compared to the previous month.

5. Add a new column to the Israeli interest rates DataFrame called Change_Percentage, representing the percentage change from the previous month.



In [12]:
# A - 1
# your code here
fed_funds_df.head(5)

,Year,Month,Current_Rate,Change_From_Last_Month
0,1993,1,3.02,NaN
1,1993,2,3.03,0.01
2,1993,3,3.07,0.04
3,1993,4,2.96,-0.11
4,1993,5,3.00,0.04


In [13]:
# A - 2
# your code here
avg_boi_interest_rate = boi_interest_rate_df['Current_Rate'].mean()
avg_boi_interest_rate

5.291062368774408

In [69]:
# A - 3
# your code here
max_increase_row = conversion_ratio_df.loc[conversion_ratio_df['Change_From_Last_Month'].idxmax()]

max_increase_year = int(max_increase_row['Year'])
max_increase_month = int(max_increase_row['Month'])
max_increase_value = max_increase_row['Change_From_Last_Month']
max_increase_year, max_increase_month, max_increase_value

(1998, 10, 0.4049999999999998)

In [17]:
# A - 4
# your code here
decrease_months = fed_funds_df[fed_funds_df['Change_From_Last_Month'] < 0].shape[0]
decrease_months

123

In [18]:
# A - 5
# your code here
boi_interest_rate_df['Change_Percentage'] = (boi_interest_rate_df['Change_From_Last_Month'] / boi_interest_rate_df['Current_Rate'].shift(1)) * 100

,Year,Month,Current_Rate,Change_From_Last_Month,Change_Percentage
0,1992,1,14.682857,NaN,NaN
1,1992,2,12.911500,-1.771357,-12.064118
2,1992,3,12.257727,-0.653773,-5.063492
3,1992,4,12.267143,0.009416,0.076813
4,1992,5,12.628947,0.361805,2.949379
...,...,...,...,...,...
388,2024,5,4.324905,0.024376,0.566824
389,2024,6,4.342760,0.017855,0.412830
390,2024,7,4.313295,-0.029465,-0.678478
391,2024,8,4.287344,-0.025951,-0.601650


## Part B: Merging DataFrames
1. Merge the US and Israeli interest rates DataFrames on Year and Month.

2. In the merged DataFrame, create a new column Rate_Difference that shows the difference between the Israeli and US interest rates for each month.

3. Identify all months when the Israeli interest rate was at least 1% higher than the US interest rate - IH (Israeli Higher).
Identify all months when the US interest rate was at least 1% higher than the Israeli interest rate - UH (US Higher).
Identify all months when the interest rates were within 1% of each other - N (Near)
make a new column called Rate_Comparison with the following values: IH, UH, or N.

4. Merge the exchange rate DataFrame with the merged interest rates DataFrame.

5. After merging all three DataFrames, find the correlation between the Rate_Difference and the Current_Rate of the exchange rate.
hint: find the connection between the difference in interest rates and the exchange rates, use any method you think is appropriate.
provide an explanation of the results (in Hebrew) - at least 2 rows of text + a proof of the connection(some output) you found.


In [48]:
# B - 1
# your code here
merged_df = pd.merge(boi_interest_rate_df, fed_funds_df, on=['Year', 'Month'], suffixes=('_Israel', '_US'))
merged_df

,Year,Month,Current_Rate_Israel,Change_From_Last_Month_Israel,Change_Percentage,Current_Rate_US,Change_From_Last_Month_US
0,1993,1,11.547143,0.657578,6.038603,3.02,NaN
1,1993,2,12.325500,0.778357,6.740690,3.03,0.01
2,1993,3,11.845000,-0.480500,-3.898422,3.07,0.04
3,1993,4,11.925333,0.080333,0.678205,2.96,-0.11
4,1993,5,12.079524,0.154190,1.292966,3.00,0.04
...,...,...,...,...,...,...,...
365,2023,6,4.557455,0.202689,4.654424,5.08,0.02
366,2023,7,4.595429,0.037974,0.833232,5.12,0.04
367,2023,8,4.655301,0.059872,1.302865,5.33,0.21
368,2023,9,4.639770,-0.015531,-0.333617,5.33,0.00


In [20]:
# B - 2
# your code here
merged_df['Rate_Difference'] = merged_df['Current_Rate_Israel'] - merged_df['Current_Rate_US']
merged_df

,Year,Month,Current_Rate_Israel,Change_From_Last_Month_Israel,Change_Percentage,Current_Rate_US,Change_From_Last_Month_US,Rate_Difference
0,1993,1,11.547143,0.657578,6.038603,3.02,NaN,8.527143
1,1993,2,12.325500,0.778357,6.740690,3.03,0.01,9.295500
2,1993,3,11.845000,-0.480500,-3.898422,3.07,0.04,8.775000
3,1993,4,11.925333,0.080333,0.678205,2.96,-0.11,8.965333
4,1993,5,12.079524,0.154190,1.292966,3.00,0.04,9.079524
...,...,...,...,...,...,...,...,...
365,2023,6,4.557455,0.202689,4.654424,5.08,0.02,-0.522545
366,2023,7,4.595429,0.037974,0.833232,5.12,0.04,-0.524571
367,2023,8,4.655301,0.059872,1.302865,5.33,0.21,-0.674699
368,2023,9,4.639770,-0.015531,-0.333617,5.33,0.00,-0.690230


In [31]:
# B - 3
# your code here
conditions = [
   merged_df['Rate_Difference'] >= 1,
   (merged_df['Rate_Difference'] > -1) & (merged_df['Rate_Difference'] < 1),
   merged_df['Rate_Difference'] <=-1
]

choices = ['IH','N','UH']

merged_df['Rate_Comparison'] = np.select(conditions, choices, default='Unknown')

In [32]:
# B - 4
# your code here
final_merged_df = pd.merge(merged_df, conversion_ratio_df, on=['Year', 'Month'])
final_merged_df

,Year,Month,Current_Rate_Israel,Change_From_Last_Month_Israel,Change_Percentage,Current_Rate_US,Change_From_Last_Month_US,Rate_Difference,Rate_Comparison,Current_Rate,Change_From_Last_Month
0,1993,1,11.547143,0.657578,6.038603,3.02,NaN,8.527143,IH,2.765,0.001
1,1993,2,12.325500,0.778357,6.740690,3.03,0.01,9.295500,IH,2.795,0.030
2,1993,3,11.845000,-0.480500,-3.898422,3.07,0.04,8.775000,IH,2.768,-0.027
3,1993,4,11.925333,0.080333,0.678205,2.96,-0.11,8.965333,IH,2.728,-0.040
4,1993,5,12.079524,0.154190,1.292966,3.00,0.04,9.079524,IH,2.729,0.001
...,...,...,...,...,...,...,...,...,...,...,...
365,2023,6,4.557455,0.202689,4.654424,5.08,0.02,-0.522545,N,3.700,-0.015
366,2023,7,4.595429,0.037974,0.833232,5.12,0.04,-0.524571,N,3.693,-0.007
367,2023,8,4.655301,0.059872,1.302865,5.33,0.21,-0.674699,N,3.801,0.108
368,2023,9,4.639770,-0.015531,-0.333617,5.33,0.00,-0.690230,N,3.824,0.023


In [39]:
# B - 5
# your code here
Correlation_fertility_difference_conversion_rate = final_merged_df['Rate_Difference'].corr(final_merged_df['Current_Rate'])
'''
הקורלציה השלילית בין ההפרש של הריביות לבין שער ההמרה
מצביע על קשר אפשרי שכאשר שיעור הריבית בישראל גבוה לעומת ארה"ב,
שער ההמרה נוטה לרדת.
אולי מכיוון שכאשר הריבית בישראל גבוהה יותר,
היקף ההשקעות בישראל עשוי לגדול,
מה שמוביל לירידה בביקוש לדולרים, מה שמוביל לכך ששער ההמרה יירד.
'''
Correlation_fertility_difference_conversion_rate

-0.20560996003882184

## Part C: Analyzing Relationships Between DataFrames
1. Analyze whether increases in US interest rates are associated with increases or decreases in the USD to ILS exchange rate in the same month.

2. Determine if changes in the Israeli interest rate have a stronger impact on the exchange rate than changes in the US interest rate.

3. Find any time lags in the effect of interest rate changes on the exchange rate. For example, does a change in US interest rates affect the exchange rate in the following month?
(כלומר- האם כשיש שינוי בריבית כלשהי - האם מייד יש שינוי בשער ההמרה? או שיש דיליי של חודש/חודשים עד שמרגישים את ההשפעה - אם בכלל?)

4. Calculate the rolling past 3-month average of the exchange rate for each data time, and add it as a new column.

5. Identify any periods where both US and Israeli interest rates were decreasing, and analyze how the exchange rate behaved during those periods.



In [43]:
# c - 1
# your code here

correlation_american = final_merged_df['Change_From_Last_Month_US'].corr(final_merged_df['Change_From_Last_Month'])
correlation_american

-0.04418886507363818

In [75]:
# c - 2
# your code here
def correlation_bigger(correlation_israeli, correlation_american):
    if abs(correlation_israeli) > abs(correlation_american):
        return f"קורלציה חזקה יותר: השפעת השיעור הריבית הישראלי על שער ההמרה (קורלציה: {correlation_israeli:.2f})"
    elif abs(correlation_israeli) < abs(correlation_american):
        return f"קורלציה חזקה יותר: השפעת השיעור הריבית האמריקאי על שער ההמרה (קורלציה: {correlation_american:.2f})"
    else:
        return f"לקורלציות השפעה שווה: {correlation_israeli:.2f}"

correlation_israeli = final_merged_df['Change_From_Last_Month_Israel'].corr(final_merged_df['Change_From_Last_Month'])
correlation_bigger(correlation_israeli, correlation_american)

'קורלציה חזקה יותר: השפעת השיעור הריבית הישראלי על שער ההמרה (קורלציה: -0.10)'

In [74]:
# c - 3
# your code here
for i in range(1, 4): 
    final_merged_df[f'US_Interest_Rate_Change_Lag_{i}'] = final_merged_df['Change_From_Last_Month_US'].shift(i)
    final_merged_df[f'Israel_Interest_Rate_Change_Lag_{i}'] = final_merged_df['Change_From_Last_Month_Israel'].shift(i)


us_correlations = {}
israel_correlations = {}

for i in range(1, 4):
    us_correlation = final_merged_df[f'US_Interest_Rate_Change_Lag_{i}'].corr(final_merged_df['Change_From_Last_Month'])
    us_correlations[f'Lag_{i}'] = us_correlation
    israel_correlation = final_merged_df[f'Israel_Interest_Rate_Change_Lag_{i}'].corr(final_merged_df['Change_From_Last_Month'])
    israel_correlations[f'Lag_{i}'] = israel_correlation


israel = [f'קורלציה ישראל עם דיליי של {lag}: {correlation:.2f}' for lag, correlation in israel_correlations.items()]
usa = [f'קורלציה ארה"ב עם דיליי של {lag}: {correlation:.2f}' for lag, correlation in us_correlations.items()]
israel, usa

קורלציה ארה"ב עם דיליי של Lag_1: 0.03
קורלציה ארה"ב עם דיליי של Lag_2: -0.02
קורלציה ארה"ב עם דיליי של Lag_3: -0.02
קורלציה ישראל עם דיליי של Lag_1: -0.08
קורלציה ישראל עם דיליי של Lag_2: -0.05
קורלציה ישראל עם דיליי של Lag_3: -0.05


(['קורלציה ישראל עם דיליי של Lag_1: -0.08',
  'קורלציה ישראל עם דיליי של Lag_2: -0.05',
  'קורלציה ישראל עם דיליי של Lag_3: -0.05'],
 ['קורלציה ארה"ב עם דיליי של Lag_1: 0.03',
  'קורלציה ארה"ב עם דיליי של Lag_2: -0.02',
  'קורלציה ארה"ב עם דיליי של Lag_3: -0.02'])

In [51]:
# c - 4
# your code here
final_merged_df['3_Month_Avg_Exchange_Rate'] = final_merged_df['Current_Rate'].rolling(window=3).mean()
final_merged_df

,Year,Month,Current_Rate_Israel,Change_From_Last_Month_Israel,Change_Percentage,Current_Rate_US,Change_From_Last_Month_US,Rate_Difference,Rate_Comparison,Current_Rate,Change_From_Last_Month,US_Interest_Rate_Change_Lag_1,US_Interest_Rate_Change_Lag_2,US_Interest_Rate_Change_Lag_3,Israel_Interest_Rate_Change_Lag_1,Israel_Interest_Rate_Change_Lag_2,Israel_Interest_Rate_Change_Lag_3,3_Month_Avg_Exchange_Rate
0,1993,1,11.547143,0.657578,6.038603,3.02,NaN,8.527143,IH,2.765,0.001,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1993,2,12.325500,0.778357,6.740690,3.03,0.01,9.295500,IH,2.795,0.030,NaN,NaN,NaN,0.657578,NaN,NaN,NaN
2,1993,3,11.845000,-0.480500,-3.898422,3.07,0.04,8.775000,IH,2.768,-0.027,0.01,NaN,NaN,0.778357,0.657578,NaN,2.776000
3,1993,4,11.925333,0.080333,0.678205,2.96,-0.11,8.965333,IH,2.728,-0.040,0.04,0.01,NaN,-0.480500,0.778357,0.657578,2.763667
4,1993,5,12.079524,0.154190,1.292966,3.00,0.04,9.079524,IH,2.729,0.001,-0.11,0.04,0.01,0.080333,-0.480500,0.778357,2.741667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,2023,6,4.557455,0.202689,4.654424,5.08,0.02,-0.522545,N,3.700,-0.015,0.23,0.18,0.08,0.062424,0.051596,0.325593,3.685333
366,2023,7,4.595429,0.037974,0.833232,5.12,0.04,-0.524571,N,3.693,-0.007,0.02,0.23,0.18,0.202689,0.062424,0.051596,3.702667
367,2023,8,4.655301,0.059872,1.302865,5.33,0.21,-0.674699,N,3.801,0.108,0.04,0.02,0.23,0.037974,0.202689,0.062424,3.731333
368,2023,9,4.639770,-0.015531,-0.333617,5.33,0.00,-0.690230,N,3.824,0.023,0.21,0.04,0.02,0.059872,0.037974,0.202689,3.772667


In [65]:
# c - 5
# your code here
def rising_or_falling(positive_changes, negative_changes, zero_changes):
    if positive_changes > negative_changes and positive_changes > zero_changes:
        return "ברוב החודשים יחס ההמרה עולה"
    elif negative_changes > positive_changes and negative_changes > zero_changes:
        return "ברוב החודשים יחס ההמרה יורד"
else:
    print("אין הקשר ברור")
final_merged_df['Both_Interest_Rates_Decreasing'] = (final_merged_df['Change_From_Last_Month_Israel'] < 0) & (final_merged_df['Change_From_Last_Month_US'] < 0)


decreasing_periods_df = final_merged_df[final_merged_df['Both_Interest_Rates_Decreasing']]

decreasing_periods_df[['Year', 'Month', 'Change_From_Last_Month_Israel', 'Change_From_Last_Month_US', 'Current_Rate', 'Change_From_Last_Month']]
positive_changes = (final_merged_df['Change_From_Last_Month_Israel'] > 0).sum()
negative_changes = (final_merged_df['Change_From_Last_Month_Israel'] < 0).sum()
zero_changes = (final_merged_df['Change_From_Last_Month_Israel'] == 0).sum()



מספר הערכים החיוביים: 166
מספר הערכים השליליים: 204
מספר הערכים האפסיים: 0
בעמודה יש רוב של ערכים שליליים


In [ ]:
# Bonus 

# Bonus
choose 3 global / local events or periods that you think could have affected the exchange rate or interest rates
and analyze the effect of these events on the data you collected.
show proof of your analysis that supports your claim.

(כלומר- תבחנו אירועים שקרו בתקופה שאתם עובדים עליה ובדקו איך הם השפיעו על הנתונים שאתם עובדים איתם)
למשל: בחירות בארה״ב, בחירות בישראל, רוסיה אוקראינה, מלחמות בישראל וכו׳.

In [ ]:
# Your code here